### Step 1. Split out Location column from "Json" format into a seperate dataset the match the main dataset

##### Seprate the Location col for parsing

In [42]:
library(readr)
library(dplyr)
library(tidyverse)
library(lubridate)
library(jsonlite)
library(data.table)
# install.packages("rjson")
library(rjson)
library(stringr)
library(tidyr)

In [15]:
df <- read.csv("retail-food-stores.csv")
names(df)

[1] "County"             "License.Number"     "Operation.Type"    
 [4] "Establishment.Type" "Entity.Name"        "DBA.Name"          
 [7] "Street.Number"      "Street.Name"        "Address.Line.2"    
[10] "Address.Line.3"     "City"               "State"             
[13] "Zip.Code"           "Square.Footage"     "Location"

In [192]:
LocTxt <- c()
Loc <- df$Location
for (i in 1: length(Loc)){
    LocTxt[i] <- gsub("\\'", '"', toString(Loc[i]))
}
Location<- as.data.frame(LocTxt, row.names = NULL, colnames = names("Location"))
tail(Location)

,LocTxt
29384,"{""longitude"": ""-74.119218"", ""needs_recoding"": False, ""human_address"": ""{""address"":""2 NEW DORP LN"",""city"":""STATEN ISLAND"",""state"":""NY"",""zip"":""10306""}"", ""latitude"": ""40.575513""}"
29385,"{""longitude"": ""-74.124557"", ""needs_recoding"": False, ""human_address"": ""{""address"":""1259 CASTLETON AVE"",""city"":""STATEN ISLAND"",""state"":""NY"",""zip"":""10310""}"", ""latitude"": ""40.633963""}"
29386,"{""longitude"": ""-74.089189"", ""needs_recoding"": False, ""human_address"": ""{""address"":""506 RICHMOND TERRACE"",""city"":""STATEN ISLAND"",""state"":""NY"",""zip"":""10301""}"", ""latitude"": ""40.646506""}"
29387,"{""longitude"": ""-74.062958"", ""needs_recoding"": False, ""human_address"": ""{""address"":""1386 BAY ST"",""city"":""STATEN ISLAND"",""state"":""NY"",""zip"":""10305""}"", ""latitude"": ""40.6098""}"
29388,"{""longitude"": ""-74.136856"", ""needs_recoding"": False, ""human_address"": ""{""address"":""1440 FOREST AVE"",""city"":""STATEN ISLAND"",""state"":""NY"",""zip"":""10302""}"", ""latitude"": ""40.624434""}"
29389,"{""longitude"": ""-74.075886"", ""needs_recoding"": False, ""human_address"": ""{""address"":""32 BAY ST"",""city"":""STATEN ISLAND"",""state"":""NY"",""zip"":""10301""}"", ""latitude"": ""40.641334""}"


##### Parse the json formed strs from Location column into different list and plot back to the data frame

In [395]:
df2 <- function(keyword, Lst){
    if (Lst != ""){
        str <- strsplit(Lst, ':|,')[[1]]
        for (i in 1: length(str)){
        
                if (grepl(keyword, str[i]) == TRUE){ 
                    value <- gsub('\\"|\\"|\\{|\\}| ', "", str[i+1])
                    
                    return (value)
                } 
            }
        }        
    else {
        value <- "N/A"
        return (value)
    }           
}     


split <- sapply(Location[,1], toString)
Logi <- lapply(split, df2, keyword = 'longitude')
Lati <- lapply(split, df2, keyword = 'latitude')
Address <- lapply(split, df2, keyword = '"address"')
City <- lapply(split, df2, keyword = 'city')
State <- lapply(split, df2, keyword = 'state')
Zip <- lapply(split, df2, keyword = 'zip')

# df2('longitude', split )
# sapply(split,  df2('longitude', split ))

##### merged the lists back to the main dataset as data frame 

In [450]:
newDf <- mutate(df, address = unlist(Address), city = unlist(City), state= unlist(State),  zip= unlist(Zip) ,longitute = unlist(lapply(Logi, toString))
, latitude = unlist(lapply(Lati, toString)))
write.table(newDf, "New_retail-food-stores.csv", sep = ",")
# ndf <- read.csv("New_retail-food-stores.csv", sep = ",")

###### data further cleaning 

* subset nessceary columns
* uniform column names
* parse Space
* parse Address columns in the format of "adree# StreenName"
* chekc out and uniform null/na values

In [84]:
clean <- function(fname) {
    d <- read.csv("New_retail-food-stores.csv")

    ## subset out the col no needed 
    df <- d[, c(-9,-10,-11,-12,-13, -15, -16)]
    ## rename all the col
    df <- rename(df, City =city, State = state, ZipCode = zip, Longitude = longitute, Latitude = latitude)
    ## parse all spaces on left and right of strings
    for (i in names(df)){
        df[, c(i)] <- sapply(df[, c(i)], trimws, which =c("both") )
    }
    df$Address <-  paste(df$Street.Number, df$Street.Name, sep = " ")
    df <- df[,c(2,3,4,5,6,9, 15,7,8,10,11,12,1, 13,14)]
    df <- df[, c(-9,-10)]
    ## to parse number out of a string :
    # regexp <- "[[:digit:]]+"
    # regexb <- "[[:alpha:]]+"
    # df$AddressNumber <- lapply(df$Address, str_extract, pattern = regexp) 
    # df$AddressStreet <- lapply(df$Address, str_extract, pattern = regexb)
    # df <- df[, c(-8)]
    # df <- transform(df,  Address = paste(df$AddressNumber, df$AddressStreet, sep = " "))

    return(df)
}
df <- clean("New_retail-food-stores.csv")
head(df)

License.Number,Operation.Type,Establishment.Type,Entity.Name,DBA.Name,Square.Footage,Address,Street.Number,State,ZipCode,County,Longitude,Latitude
720644,Store,JAC,SPEEDWAY LLC,SPEEDWAY #07708,2500,601 SARATOGA ST,601,NY,12047,Albany,-73.700539,42.754424
734310,Store,A,GREEN ISLAND MART INC,GREEN ISLAND MART,1800,165 PAINE STREET,165,NY,12183,Albany,-73.691984,42.746003
720557,Store,JAC,"SAEED SADIQ, SAIKA NOREEN",19 STREET QUICK STOP,1200,315 19TH STREET,315,NY,12189,Albany,-73.703443,42.73063
724747,Store,JAC,SUNOCO RETAIL LLC,SUNOCO #2706,3000,1465 WASHINGTON AVE,1465,NY,12203,Albany,-73.821889,42.690493
730927,Store,JAC,NAPREDAK LLC,KNOX STORE,700,2160 BERNE ALTAMONT RD,2160,NY,12107,Albany,,
720578,Store,A,ADIRONDACK S AND S LLC,NEWSSTAND,600,1 CROSSGATES MALL RD D 105,1,NY,12203,Albany,-73.850605,42.687038


In [52]:
head(df[which(df$Longitude == "N/A"),])

,County,License.Number,Operation.Type,Establishment.Type,Entity.Name,DBA.Name,Street.Number,Street.Name,Square.Footage,City,State,ZipCode,Longitude,Latitude
113,Albany,727648,Store,A,DONIETERESA,DMVNEWS,,NYSDMVCORE3SOUTHSWANSTREET#,300,ALBANY,NY,N/A,N/A,N/A
218,Albany,10514,Store,A,LINDT&SPRUNGLIUSAINC,LINDTCHOCOLATE333,,ONECROSSGATESMALLRDSUITEP-21,1500,ALBANY,NY,N/A,N/A,N/A
229,Albany,15821,Store,JACX,TESTESTABLISHMENT,TESTRECORD,,TEST,12345,TEST,NY,N/A,N/A,N/A
232,Albany,10618,Store,A,SAEEDKHALID,TOWERNEWS,,EMPIRESTATEPLAZACORNING,550,ALBANY,NY,N/A,N/A,N/A
276,Albany,10448,Store,A,CVSALBANYLLC,CVSPHARMACY05037,,RT9WFAITHPLAZA,4500,RAVENA,NY,N/A,N/A,N/A
290,Albany,10723,Store,A,AGOSTINIANGIE,ANGELSLASTINGIMPRESSI,,AGY3&4EMPIRESTATEPLAZA,400,ALBANY,NY,N/A,N/A,N/A


In [85]:
write.table(df, "retail-food-stores_tidyData.csv", sep = ",")